# 有模型数值迭代

假设动力系统完全已知的情况下，用迭代的数值方法来求解Bellman方程，得到价值函数与最优策略。

由于有模型迭代没有从数据里学习，所以<u>一般不认为是一种机器学习或强化学习方法</u>

## 度量空间与压缩映射

**有模型策略迭代的理论基础**：度量空间上的Banach不动点定理

### 度量空间及其完备性

【**度量**】定义在集合上的二元函数。对于集合$X$，其上度量$d:X \times X \to R$，需要满足：
* **非负性**：对于任意$x',x'' \in X$，有$d(x',x'') \ge 0$
* **同一性**：对于任意$x',x'' \in X$，若$d(x',x'') = 0$，$x' = x''$
* **对称性**：对于任意$x',x'' \in X$，有$d(x',x'') = d(x'',x')$
* **三角不等式**：对于任意$x',x'',x''' \in X$，有$d(x',x'') \le d(x',x''') + d(x'',x''') $

有序对$(X,d)$又称为**度量空间**

> 考虑有限Markov决策过程状态函数$v(s)$，其所有可能的取值组成集合$V \in R^{|S|}$，定义$d_{\infty}$：$d_{\infty} = \max_{s \in S} |v'(s) - v''(s)|$。由于满足非负性、同一性、对称性、三角不等式，所以$d_{\infty}$是$V$上的一个度量

【**完备性**】对于一个度量空间，若Cauchy序列都收敛在该空间内，则称这个度量空间是**完备的**

> 实数集就是著名的完备空间。

### 压缩映射与Bellman算子

【**压缩映射**】对于一个度量空间$(X,d)$和其上的一个映射$t:X \to X$，如果存在某个实数$\gamma \in (0,1)$，使得对于任意的$x',x''\in X$，都有：$d(t(x'),t(x'')) < \gamma d(x',x'')$，则称$t$为**压缩映射**。$\gamma$为Lipschitz常数。

【**Bellman最优算子**】
* 给定策略$\pi(a|s)$的**Bellman期望算子**$t_\pi:V \to V$：
  $$
  t_\pi(v)(s) = \sum_a \pi(a|s) \left[ r(s,a) + \gamma \sum_{s'} p(s'|s,a)v(s') \right]
  $$
  
* **Bellman最优算子**$t_*:V \to V$：
  $$
  t_*(v)(s) = \max_{a\in A} \left[ r(s,a) +\gamma \sum_{s' \in S} p(s'|s,a)v_*(s') \right]
  $$

这两个算子都是压缩映射。证明略

### Banach不动点定理

【**不动点**】对于度量空间$(X,d)$上的映射$t:X \to X$，若$x \in X$使得$t(x) = x$，则称$x$是映射$t$的**不动点**

> 策略$\pi$的状态价值函数$v_\pi(s)$满足Bellman期望方程，是Bellman期望算子$t_\pi$算子的不动点。
>
> 最优状态价值$v_*(s)$满足Bellman最优方程，是Bellman最优算子$t_*$的不动点

【**Banach不动点定理**】

* 又称为压缩映射定理。$(X,d)$是非空的完备度量空间，$t:X \to X$是一个压缩映射，则映射$t$在$X$内有且仅有一个**不动点**。

* 此不动点可通过下列方法求出：从$X$内任意的一个元素$x_0$开始，定义迭代序列$x_k = t(x_{k-1})$，这个序列收敛，且极限为$x_{+\infty}$

* 从任意的起点开始，不断迭代使用压缩映射，就能得到不动点。迭代正比于$\gamma^k$的速度收敛。（$k$为迭代次数）
* **由此可以利用迭代法求得策略的价值或最优价值**。

## 有模型策略迭代

在给定动力系统$p$的情况下的策略评估、策略改进和策略迭代

* **策略评估**：对于给定策略$\pi$，<u>估计策略的价值，包括动作价值和状态价值</u>
* **策略改进**：对于给定策略$\pi$，在已知其价值函数的情况下<u>找到一个更优策略</u>
* **策略迭代**：综合利用策略评估和策略改进，找到最优策略

### 策略评估

#### 算法一

【输入】动力系统$p$，策略$\pi$

【输出】状态价值函数$v_\pi$的估计值

【参数】控制迭代次数的参数（如容忍误差度$\varepsilon_{\max}$或最大迭代次数$k_{\max}$）

1. **初始化**：对于$s \in S$，将$v_0(s)$初始化为任意值（比如0）。如果有终止状态，则将终止状态初始化为0

2. **迭代**：对于$k \leftarrow 1,2,3\dots$，迭代执行以下策略：

   1. 对于$s \in S$，逐一更新$v_{k+1}(s) \leftarrow \sum_{a} \pi(a|s)q_k(s,a)$，其中
      $$
      q_k(s,a) \leftarrow r(s,a) + \gamma \sum_{s'}p(s'|s,a)v_k(s')
      $$

   2. 若满足迭代终止条件，则跳出循环

      **常用终止条件**：1. 达到最大迭代次数$k_{\max}$；2. 满足最大误差限$\varepsilon_{\max}$达到精度要求

#### 算法二

【输入】动力系统$p$，策略$\pi$

【输出】状态价值函数$v_\pi$的估计值

【参数】控制迭代次数的参数（如容忍误差度$\varepsilon_{\max}$或最大迭代次数$k_{\max}$）

1. **初始化**：对于$s \in S$，将$v_0(s)$初始化为任意值（比如0）。如果有终止状态，则将终止状态初始化为0
2. **迭代**：对于$k \leftarrow 1,2,3\dots$，迭代执行以下策略：
   1. 计算新的状态价值$v_{新} \leftarrow \sum_{a} \pi(a|s)\left[ r(s,a) + \gamma \sum_{s'}p(s'|s,a)v(s') \right]$
   2. 对于使用误差限的情况，更新本次迭代观测到的最大误差$\varepsilon \leftarrow \max \{ \varepsilon | v_{新} - v(s) \}$
   3. 更新状态价值函数$v(s) \leftarrow v_{新}$
3. 满足迭代终止条件则退出循环

【**迭代策略评估算法的意义**】

1. 此策略评估算法将作为策略迭代算法的一部分，可用于最优策略的求解
2. 在此基础上修改可得到迭代求解最优策略的算法

### 策略改进

【**策略改进定理**】对于策略$\pi$和$\pi'$，若$v_\pi \le \sum_a \pi'(a|s)q_\pi(s,a)$，则$\pi \le \pi'$，即$v_\pi \le v_{\pi’}$（严格状态下全部为小于号）

* 对于一个确定性策略，若存在着$s \in s, a\in A$，使得$q_\pi(s,a) > v_\pi(s)$，那么我们可以构造一个新的确定策略$\pi'$，它在状态$s$做动作$a$，而在除状态$s$以外的状态的动作都和策略$\pi$一样。由此得到比$\pi$更好的策略$\pi'$

#### 算法一

【输入】动力系统$p$，策略$\pi$及其状态价值函数$v_\pi$

【输出】改进的策略$\pi'$，或策略$\pi$已经达到最优的标志

1. 对每个状态$s\in S$，执行以下操作：
   1. 为每个动作$a \in A$求得动作价值函数$q_\pi(s,a) \leftarrow r(s,a) + \gamma\sum_{s'} p(s'|s,a)v_\pi(s')$
   2. 找到使得$q_\pi(s,a)$最大动作$a$，即$\pi'(s) = \mathop{\arg\max}_aq(s,a)$
2. 如果更新所得策略和旧策略相同，则说明就策略已是最优；否则输出改进策略

#### 算法二（节约空间）

【输入】动力系统$p$，策略$\pi$及其状态价值函数$v_\pi$

【输出】改进的策略（仍存储为$\pi$），或策略$\pi$已经达到最优的标志$o$

1. 初始化$o \leftarrow \mathrm{True}$
2. 对每个状态$s\in S$，执行以下操作：
   1. 为每个动作$a \in A$求得动作价值函数$q_\pi(s,a) \leftarrow r(s,a) + \gamma\sum_{s'} p(s'|s,a)v_\pi(s')$
   2. 找到使得$q(s,a)$最大的动作$a'$，即$a' = \mathop{\arg\max}_a q(s,a)$
   3. 如果$\pi(s) \ne a'$，则更新$\pi(s) \leftarrow a', o \leftarrow \mathrm{False}$

### 策略迭代

【**策略迭代**】综合利用策略评估和策略改进求解最优策略的迭代方法

* 从任意确定性策略$\pi_0$开始，交替进行策略评估和策略改进。
  * 这里的策略改进是严格的策略改进，即改进前后的策略不同。
* 对于状态空间和动作空间均有限的Markov决策过程，其<u>可能的确定性策略数有限</u>
* 在迭代过程中得到的策略序列$\pi_0,\pi_1,\dots$一定可收敛，使得某个$k$，有$\pi_k=\pi_{k+1}$（即对任意的$s \in S$均有$\pi_{k+1}(s) = \pi_k(s)$）。
* 由于$\pi_k=\pi_{k+1}$时有$\pi_{k+1}(s) = \pi_k(s) = \mathop{\arg\max}_a q_{\pi_k}(s,a)$，进而$v_{\pi_k(s)} = \max_aq_{\pi_k}(s,a)$，满足Bellman最优方程

#### 算法一

【输入】动力系统$p$

【输出】最优策略

1. 初始化：将策略$\pi_0$初始化为任意的确定性策略
2. 迭代：对于$k \leftarrow 1,2,3\dots$，执行以下步骤：
   1. 策略评估：使用策略评估算法，计算策略$\pi_k$的状态价值函数$v_{\pi_k}$
   2. 策略更新：利用状态价值函数$v_{\pi_k}$改进确定性策略$\pi_k$，得到改进的确定性策略$\pi_{k+1}$。若$\pi_k=\pi_{k+1}$则迭代完成，返回$\pi_k$为最优策略

#### 算法二

【输入】动力系统$p$

【输出】最优策略

【参数】策略评估所需参数

1. 初始化：将策略$\pi$初始化为任意的确定性策略
2. 迭代：执行以下步骤
   1. 策略评估：使用策略评估算法，计算策略$\pi$的状态价值函数并存在$v$中
   2. 策略更新：利用$v$中存储的价值函数进行策略改进，并将改进后策略存在$\pi$中。若本次策略改进算法指示当前策略$\pi$已经是最优策略，则迭代完成，返回$\pi$为最优策略

## 有模型价值迭代

【**价值迭代**】利用迭代求解最优价值函数进而求解最优策略的方法。

#### 算法一

【输入】动力系统$p$

【输出】最优策略估计$\pi$

【参数】策略评估所需参数

1. 初始化：$v_0(s) \leftarrow$任意值，$s \in S$。若有终止状态，$v_0(s_{终止}) \leftarrow 0$

2. 迭代：对于$k \leftarrow 1,2,3\dots$，执行以下步骤：

   1. 对于$s  \in S$，逐一更新$v_{k+1}(s)\leftarrow \max_a \left\{ r(s,a) +\gamma\sum_{s'}p(s'|s,a)v_{k+1}(s') \right\}$
   2. 若满足最大误差限或达到最大迭代次数则跳出循环

3. 根据价值函数输出确定性策略$\pi_*$，使得
   $$
   \pi_*(s) = \mathop{\arg \max}_a \{r(s,a) +\gamma\sum_{s'}p(s'|s,a)v_{k+1}(s')\}
   $$
   

#### 算法二（节约空间）

【输入】动力系统$p$

【输出】最优策略估计$\pi$

【参数】策略评估所需参数

1. 初始化：$v_0(s) \leftarrow$任意值，$s \in S$。若有终止状态，$v_0(s_{终止}) \leftarrow 0$

2. 迭代：对于$k \leftarrow 1,2,3\dots$，执行以下步骤：

   1. 对于使用误差限的情况，初始化本次迭代观测到的最大误差$\varepsilon \leftarrow 0$
   2. 对于$s  \in S$，执行以下步骤：
      1. 计算新状态价值$v_{新} \leftarrow \max_a \left\{ r(s,a) +\gamma\sum_{s'}p(s'|s,a)v_{k+1}(s') \right\}$
      2. 对于使用误差限的情况，更新本次迭代观测到的最大误差$\varepsilon \leftarrow \max\{\varepsilon,|v_{新}-v(s)|\}$
      3. 更新状态价值函数$v(s) \leftarrow v_{新}$
   3. 如果满足误差限或达到最大迭代次数则跳出循环

3. 根据价值函数输出确定性策略：
   $$
   \pi(s) = \mathop{\arg \max}_a \{r(s,a) +\gamma\sum_{s'}p(s'|s,a)v_{k+1}(s')\}
   $$

## 动态规划

### 从动态规划看迭代算法

【**动态规划**】

* 将原问题分解为若干个子问题，若知道子问题的解，就很容易知道原问题的解
* 分解得到多个子问题，有许多子问题的相同的，不需要重复计算

求解Bellman期望方程和Bellman最优方程的迭代算法实践了动态规划的思想。在第$k$此迭代过程中，计算$v_{k+1}(s)$中的每一个值，都需要用到的数值。但是，考虑到求解$v_{k+1}$各个元素时使用了相同的$v_k$值，因此不用重复计算$v_k$

【**自益**】

* 在求解过程中$v_k$和$v_{k+1}$都是$v$的估计值。用一个估计值来估计另一个估计值的做法称为自益

实际求解过程中直接采用这样的动态规划。

### 异步动态规划

【**思想**】每次扫描不再完整地更新一整套状态价值函数，知识更新部分感兴趣的值

> 例如：有些状态$s$不会转移到另外一些状态（例如对任意$a \in A$均有$p(s'|s,a) = 0$的状态$s'$），那么更新状态$s$的价值函数后再更新$s'$没有意义。通过只做有意义的更新可能会大大减少计算量

【**优先更新**】

* 根据Bellman误差来选择性更新状态的算法。

* 在迭代过程中，当更新一个状态后，试图找到一个Bellman误差最大的状态并更新该状态。

* 当更新一个状态函数后，针对这个状态的价值函数会影响到的状态价值函数，计算Bellman误差
  $$
  \left| \max_a \left( r(s,a) + \gamma\sum_{s'} p(s'|s,a)v(s') \right) - v(s) \right|
  $$

## 案例：冰面滑行 FrozenLake-v0

In [ ]:
import numpy as np
np.random.seed(0)
import gym

### 实验环境使用

- 16个不同状态
- 4个不同动作
- 动力系统储存在`env.P`中，`env.unwrapped.P[14][2]`可查看状态14中动作2的动力。它是一个元组列表，每个元组包括概率、下一状态、奖励值、回合值结束这四个部分。

In [ ]:
env = gym.make('FrozenLake-v0')
env.seed(0)
print("观测空间 = {}".format(env.observation_space))
print("动作空间 = {}".format(env.action_space))
print("观测空间大小 = {}".format(env.unwrapped.nS))
print("动作空间大小 = {}".format(env.unwrapped.nA))
env.unwrapped.P[14][2]  # 查看动力

用策略执行一回合

In [ ]:
def play_policy(env, policy, render=False):
    total_reward = 0
    observation = env.reset()
    while True:
        if render:
            env.render()
        action = np.random.choice(env.action_space.n,
                p=policy[observation])
        observation,reward,done,_ = env.step(action)
        total_reward += reward
        if done:
            break
    return total_reward

随机策略

In [ ]:
random_policy = np.ones((env.unwrapped.nS, env.unwrapped.nA)) / env.unwrapped.nA

episode_rewards = [play_policy(env, random_policy)  for _ in range(100)]
print("随机策略 平均奖励：{}".format(np.mean(episode_rewards)))

### 策略评估

In [ ]:
def v2q(env, v, s=None, gamma = 1.): # 根据状态价值函数计算动作价值函数
    if s is not None: # 针对单个状态求解
        q = np.zeros(env.unwrapped.nA)
        for a in range(env.unwrapped.nA):
            for prob, next_state, reward, done in env.unwrapped.P[s][a]:
                q[a] += prob * \
                        (reward + gamma * v[next_state] * (1. - done))
    else: # 针对所有状态求解
        q = np.zeros((env.unwrapped.nS, env.unwrapped.nA))
        for s in range(env.unwrapped.nS):
            q[s] = v2q(env, v, s, gamma)
    return q

def evaluate_policy(env, policy, gamma=1., tolerant=1e-6):
    v = np.zeros(env.unwrapped.nS) # 初始化状态价值函数
    while True: # 循环
        delta = 0
        for s in range(env.unwrapped.nS):
            vs = sum(policy[s] * v2q(env, v, s, gamma)) # 更新状态价值函数
            delta = max(delta, abs(v[s]-vs)) # 更新最大误差
            v[s] = vs # 更新状态价值函数
        if delta < tolerant: # 查看是否满足迭代条件
            break
    return v  

评估随机策略的价值函数:

In [ ]:
print('状态价值函数：')
v_random = evaluate_policy(env, random_policy)
print(v_random.reshape(4, 4))

print('动作价值函数：')
q_random = v2q(env, v_random)
print(q_random)

### 策略改进

In [ ]:
def improve_policy(env, v, policy, gamma=1.):
    optimal = True
    for s in range(env.unwrapped.nS):
        q = v2q(env, v, s, gamma)
        a = np.argmax(q)
        if policy[s][a] != 1.:
            optimal = False
            policy[s] = 0.
            policy[s][a] = 1.
    return optimal

对随机策略进行改进

In [ ]:
policy = random_policy.copy()
optimal = improve_policy(env, v_random, policy)
if optimal:
    print('无更新，最优策略为：')
else:
    print('有更新，更新后的策略为：')
print(policy)

### 策略迭代

In [ ]:
def iterate_policy(env,gamma=1.,tolerant=1e-6):
    # 初始化为任一策略
    policy = np.ones((env.unwrapped.nS, env.unwrapped.nA)) / env.unwrapped.nA
    while True:
        v = evaluate_policy(env,policy,gamma,tolerant)
        if improve_policy(env,v,policy):
            break
        return policy,v

In [ ]:
policy_pi, v_pi = iterate_policy(env)
print('状态价值函数 =')
print(v_pi.reshape(4, 4))
print('最优策略 =')
print(np.argmax(policy_pi, axis=1).reshape(4, 4))

测试策略

In [ ]:
episode_rewards = [play_policy(env, policy_pi)  for _ in range(100)]
print("策略迭代 平均奖励：{}".format(np.mean(episode_rewards)))

### 价值迭代

In [ ]:
def iterate_value(env,gamma=1.,tolerant=1e-6):
    v = np.zeros(env.unwrapped.nS)
    while True:
        delta = 0
        for s in range(env.unwrapped.nS):
            vmax = max(v2q(env,v,s,gamma))
            delta = max(delta,abs(v[s] - vmax))
            vs = vmax
        if delta < tolerant:
            break

    policy = np.zeros((env.unwrapped.nS, env.unwrapped.nA))
    for s in range(env.unwrapped.nS):
        a = np.argmax(v2q(env,v,s,gamma))
        policy[s][a] = 1.
    return policy, v

In [ ]:
policy_vi, v_vi = iterate_value(env)
print('状态价值函数 =')
print(v_vi.reshape(4, 4))
print('最优策略 =')
print(np.argmax(policy_vi, axis=1).reshape(4, 4))

测试策略

In [ ]:
episode_rewards = [play_policy(env, policy_vi) for _ in range(100)]
print("价值迭代 平均奖励：{}".format(np.mean(episode_rewards)))